In [2]:
import pandas as pd
data = pd.read_csv('yellow_cab_2017.csv.gz',  compression='gzip',
                   error_bad_lines=False)

In [ ]:
data.columns

In [ ]:
first_grouping_2017=data.groupby(['time_block', 'PULocationID', 'DOLocationID'],  as_index=False).agg({'passenger_count': ['count', 'sum'], 'trip_distance': 'mean', 'tip_amount': 'sum', 'tolls_amount': 'sum', 'total_amount': 'sum',   'time_delta':'mean' })

first_grouping_2017.to_csv('first_grouping_2017.csv', encoding='utf-8')

In [ ]:
demand_data_2017=first_grouping_2017.groupby(['time_block', 'PULocationID'],  as_index=False).agg({list(first_grouping_2017)[3]: 'sum', list(first_grouping_2017)[4]: 'sum', list(first_grouping_2017)[6]: 'sum',  list(first_grouping_2017)[7]: 'sum', list(first_grouping_2017)[8]: 'sum' })

demand_data_2017.to_csv('demand_data_2017.csv', encoding='utf-8')

In [ ]:
!gzip demand_data_2017.csv

In [ ]:
data=data[(data['trip_distance'] > 0) & (data['trip_distance'] < 5 ) & (data['time_delta'] > 0 ) & (data['time_delta'] < 1800 ) ]

In [ ]:
first_grouping_for_neighbors_2017=data.groupby(['time_block', 'PULocationID', 'DOLocationID'],  as_index=False).agg({'passenger_count': ['count', 'sum'], 'trip_distance': 'mean', 'tip_amount': 'sum', 'tolls_amount': 'sum', 'total_amount': 'sum',   'time_delta':'mean' })


In [3]:
adj = pd.read_csv('adjacency_2017.csv', index_col=0)
display(adj.head())
whitelist = adj.columns.tolist()

,24,151,238,239,143,142,50,48,246,68,...,233,163,229,141,140,237,43,236,263,262
24,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
151,1,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
238,0,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
239,0,0,1,0,1,1,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
143,0,0,0,1,0,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
adj_dict = {int(x):[int(y) for z,y in zip(adj[x], whitelist) if z == 1] for x in whitelist}

# include self in adjacency list
# adj_dict = {x:y+[x] for x,y in adj_dict.items()}

adj_dict

{24: [151, 43],
 151: [24, 238, 43],
 238: [151, 239, 43],
 239: [238, 143, 142, 43],
 143: [239, 142, 50],
 142: [239, 143, 48, 163, 43],
 50: [143, 48, 246],
 48: [142, 50, 246, 68, 100, 230, 163],
 246: [50, 48, 68, 158],
 68: [48, 246, 158, 249, 90, 186, 100],
 158: [246, 68, 249, 125],
 249: [68, 158, 125, 113, 114, 90],
 125: [158, 249, 211, 231, 114],
 211: [125, 231, 144, 114],
 231: [125, 211, 13, 261, 209, 45, 144],
 13: [231, 261, 12],
 261: [231, 13, 12, 88, 87, 209],
 12: [13, 261, 88],
 88: [261, 12, 87],
 87: [261, 88, 209],
 209: [231, 261, 87, 45],
 45: [231, 209, 148, 144, 232],
 148: [45, 144, 232, 79],
 144: [211, 231, 45, 148, 114],
 232: [45, 148, 4],
 4: [232, 79, 224],
 79: [148, 4, 113, 114, 107, 224],
 113: [249, 79, 114, 234],
 114: [249, 125, 211, 144, 79, 113],
 90: [68, 249, 234, 186],
 234: [113, 90, 107, 164, 186],
 107: [79, 234, 224, 137, 170],
 224: [4, 79, 107, 137],
 137: [107, 224, 170, 233],
 170: [107, 137, 164, 161, 162, 233],
 164: [234, 170, 1

In [ ]:
first_grouping_for_neighbors_2017

In [ ]:
to_iter_over = list(zip(first_grouping_for_neighbors_2017['time_block'], first_grouping_for_neighbors_2017['PULocationID'], first_grouping_for_neighbors_2017['DOLocationID']))

In [ ]:
first_grouping_for_neighbors_2017.dtypes

In [ ]:
to_iter_over[0]

In [ ]:
import numpy as np
matching_groups = [adj_dict.get(pick_zip) is not None and drop_zip in adj_dict.get(pick_zip) for __,pick_zip,drop_zip in to_iter_over]

# print the number of adjacencies present
np.sum(matching_groups)

In [ ]:
adj_grouping_for_2017 = first_grouping_for_neighbors_2017[matching_groups]#.to_csv('')

In [ ]:
___, month, day_of_month, hour = zip(*adj_grouping_for_2017['time_block'].apply(lambda x: x.replace(' ','/').split('/')))
#adj_grouping_for_2017['time_block'].head().apply(lambda x: x.replace(' ','/').split('/'))

In [ ]:
month = list(month)
hour = list(hour)
day_of_month = list(day_of_month)

display(month[0:5])
display(hour[0:5])
display(day_of_month[0:5])

In [ ]:
adj_grouping_for_2017['month'] = month
adj_grouping_for_2017['hour'] = hour
adj_grouping_for_2017['day_of_month'] = day_of_month

In [ ]:
adj_grouping_for_2017.head()

In [ ]:
from datetime import datetime as dt

dow = adj_grouping_for_2017['time_block'].apply(lambda x: dt.strptime(x, '%Y/%m/%d %H').weekday())
#display(adj_grouping_for_2017.head()['time_block'].apply(lambda x: dt.strptime(x, '%Y/%m/%d %H')))

In [ ]:
np.max(dow)

In [ ]:
adj_grouping_for_2017['day_of_week'] = dow

In [ ]:
pu_do = adj_grouping_for_2017['PULocationID'].apply(lambda x: str(x)+'_') + adj_grouping_for_2017['DOLocationID'].apply(lambda x: str(x))
#adj_grouping_for_2017['PU_DO'] = 

In [ ]:
adj_grouping_for_2017['PU_DO'] = pu_do

In [ ]:
adj_grouping_for_2017.head()

In [ ]:
year_mask = adj_grouping_for_2017['time_block'].apply(lambda x: '2017' in x)
year_mask

In [ ]:
adj_grouping_for_2017 = adj_grouping_for_2017[year_mask]


In [ ]:
adj_grouping_for_2017.to_csv('neighbors_data_2017_fixed.csv', encoding='utf-8')